# SOLUZIONE GREEDY

## Import

In [1]:
import math

## Funzioni per la gestione dell'input e dell'output

In [1]:
def read_input(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()

    R, S, U, P, M = map(int, lines[0].strip().split())
    unavailable_slots = []

    for i in range(1, U + 1):
        unavailable_slots.append(tuple(map(int, lines[i].strip().split())))

    servers = []
    for i in range(U + 1, U + 1 + M):
        servers.append(tuple(map(int, lines[i].strip().split())))

    return R, S, U, P, M, unavailable_slots, servers

def write_output(datacenter, file_path, score, score_path):
    datacenter.servers.sort(key=lambda x: x.number, reverse=False)
    with open(file_path, 'w') as f:
        for srv in datacenter.servers:
            if srv == 'x':
                f.write('x\n')
            else:
                f.write(f"{srv.row} {srv.slot} {srv.pool}\n")
    with open(score_path, 'w') as s:
        s.write(f"{score}\n")

## Lettura dei file di input

In [2]:
#path = "input_output/hashcode_2015_qualification_round.in"
path = "input_output/input.txt"
R, S, U, P, M, unavailable_slots, serversData = read_input(path)

## Modellazione dei Server

In [3]:
class Server(object):

    def __init__(self, number, size, capacity):
        self.number = number
        self.size = size
        self.capacity = capacity
        self.slot = -1
        self.row = -1
        self.pool = -1

    def __str__(self):
        return 'size: %d, cap: %d, ratio: %.02f, (%d,%d), pool=%d' % (
            self.size, self.capacity, self.ratio, self.slot, self.row,
            self.pool)

    @property
    def ratio(self):
        return float(self.capacity) / float(self.size) #Calcolo della densità




## Modellazione del Datacenter

In [4]:
class DataCenter(object):

    def __init__(self, R, S, unavailable_slots, P, M, servers):
        self.R = R
        self.S = S
        self.unavailable_slots = unavailable_slots
        self.P = P
        self.M = M
        self.grid = [[0 for _ in range(S)] for _ in range(R)]
        self.servers = servers
        self.servers.sort(key=lambda x: x.ratio, reverse=True)
        self.pools_capacity = [0 for _ in range(P)]

    def init_grid(self): #Inizializzazione della griglia con gli slot non disponibili
        for r, s in self.unavailable_slots:
            self.grid[r][s] = -1

    def put_servers(self): #Assegnamento dei server alla griglia in ordine di densità decrescente rispettando i vincoli
        for i in range(self.M):
            for r in range(self.R):
                for s in range(self.S):
                    if self.servers[i].row != -1:  #server già piazzato
                        break
                    if self.grid[r][s] != 0: #slot già assegnato
                        continue
                    if s + self.servers[i].size > self.S:  #server troppo grande per lo spazio disponibile nella riga
                        continue
                    can_place = True
                    for offset in range(self.servers[i].size): #assicura che gli slot successivi siano liberi per la grandezza del server i
                        if self.grid[r][s + offset] != 0:
                            can_place = False
                            break
                    if not can_place:
                        continue
                    print(f"Il server {i} con ratio {self.servers[i].ratio} assegnato a ({r},{s})")
                    self.servers[i].row = r
                    self.servers[i].slot = s
                    for offset in range(self.servers[i].size): #occupa gli slot assegnati al server i
                        self.grid[r][s + offset] = 1  
                    break

    def server_to_pools(self):
        min_capacity = 1000
        for i in range(self.M):
            if self.servers[i].pool != -1: #server già assegnato a un pool 
                    break
            for p in range(self.P): #trova il server con la minore capacità
                if self.pools_capacity[p] <= min_capacity:
                    min_pool = p
                    min_capacity = self.pools_capacity[p] + servers[i].capacity

            #Assegna il server al ppol con la minore capacità in modo da bilanciare i pool
            self.servers[i].pool = min_pool
            self.pools_capacity[min_pool] += self.servers[i].capacity
            print(f"Il server {i} asssegnato al pool {min_pool}")
            
    def minimum_guaranteed_capacity(self): #Calcola la capacità minima garantita per ogni pool
        z = [0 for p in range(self.P)]
        for p in range (self.P):
            c_tot = sum(servers[i].capacity for i in range(self.M) if servers[i].pool == p)
            for r in range(self.R):
                c_row = sum(servers[i].capacity for i in range(self.M) if servers[i].pool == p and servers[i].row == r)
            z[p] = c_tot - c_row
            print(f"Il pool {p} ha capacità minima garantita {z[p]}")
        return z

## Creazione del datacenter

In [5]:
servers = [Server(i, size, capacity) for i, (size, capacity) in enumerate(serversData)]
datacenter = DataCenter(R, S, unavailable_slots, P, M, servers)
datacenter.init_grid()
datacenter.put_servers()

Il server 0 con ratio 15.0 assegnato a (0,0)
Il server 1 con ratio 10.0 assegnato a (0,1)
Il server 2 con ratio 6.666666666666667 assegnato a (1,0)
Il server 3 con ratio 5.0 assegnato a (1,3)


## Allocazione dei server ai pool

In [6]:
datacenter.server_to_pools()
for p in range(P):
    print(f"Il pool {p} ha capacità {datacenter.pools_capacity[p]}")
score = min(datacenter.minimum_guaranteed_capacity())       

Il server 0 asssegnato al pool 1
Il server 1 asssegnato al pool 0
Il server 2 asssegnato al pool 1
Il server 3 asssegnato al pool 0
Il pool 0 ha capacità 20
Il pool 1 ha capacità 35
Il pool 0 ha capacità minima garantita 10
Il pool 1 ha capacità minima garantita 15


## Elaborazione dell'output

In [7]:
write_output(datacenter, "input_output/outputGreedy.txt", score, "input_output/score_greedy.txt")
print(f"Score: {score}")

Score: 10
